# Lesson 4: Building a Multi-Document Agent

## Setup

In [12]:
USE_OPENAI = False  # True

In [13]:
import os
import nest_asyncio
nest_asyncio.apply()

In [14]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

if USE_OPENAI:
    Settings.llm = OpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
else:
    Settings.llm = Ollama(model="llama3:instruct", request_timeout=120.0)
    Settings.embed_model = OllamaEmbedding(
        model_name="llama3:instruct",
        base_url="http://localhost:11434",
        ollama_additional_kwargs={"mirostat": 0})

Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

## 1. Setup an agent over 3 papers

**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [15]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "../data/metagpt.pdf",
    "../data/longlora.pdf",
    "../data/selfrag.pdf",
]

In [16]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: ../data/metagpt.pdf
Getting tools for paper: ../data/longlora.pdf
Getting tools for paper: ../data/selfrag.pdf


In [17]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [18]:
llm = Settings.llm

In [19]:
len(initial_tools)

6

In [20]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [21]:
response = agent.query(
    "Tell me about the evaluation dataset used in LongLoRA, "
    "and then tell me about the evaluation results"
)

Added user message to memory: Tell me about the evaluation dataset used in LongLoRA, and then tell me about the evaluation results
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation dataset"}
=== Function Output ===
PG19
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation results"}
=== Function Output ===
The evaluation results include reporting perplexity for models and baselines on proof-pile (Azerbayev et al., 2022) and PG19 datasets. The models achieve better perplexity with longer context sizes, indicating the effectiveness of the fine-tuning method. The perplexity decreases as the context size increases, with improvements observed when increasing the context window size. Additionally, the maximum context length that can be fine-tuned on a single 8 × A100 machine is examined, showing promising results on extremely large settings. Some perplexity degradation is noted on small context sizes fo

In [22]:
response = agent.query("Give me a summary of both Self-RAG and LongLoRA")
print(str(response))

Added user message to memory: Give me a summary of both Self-RAG and LongLoRA
=== Calling Function ===
Calling function: summary_tool_selfrag with args: {"input": "Self-RAG"}
=== Function Output ===
Self-RAG is a framework that enhances the quality and factuality of large language models by training a language model to learn to retrieve, generate, and critique text passages and its own generation through the use of reflection tokens. It involves a Critic LM and a Generator LM to evaluate text based on these reflection tokens, predicting whether external information retrieval is necessary and generating responses accordingly. The system aims to ensure that responses are relevant, supported by evidence, and useful in answering queries, with human evaluations indicating that Self-RAG outputs are often plausible, supported by relevant passages, and aligned with the predicted reflection tokens. The system's performance can be impacted by the scale of training data and the accuracy of predic

## 2. Setup an agent over 11 papers

### Download 11 ICLR papers

In [23]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf"
]

To download these papers, below is the needed code:


    #for url, paper in zip(urls, papers):
         #!wget "{url}" -O "{paper}"
    
    
**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [24]:
# from utils import get_doc_tools
# from pathlib import Path

# paper_to_tools_dict = {}
# for paper in papers:
#     print(f"Getting tools for paper: {paper}")
#     vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
#     paper_to_tools_dict[paper] = [vector_tool, summary_tool]

### Extend the Agent with Tool Retrieval

In [25]:
all_tools = initial_tools  # [t for paper in papers for t in paper_to_tools_dict[paper]]

In [26]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [27]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [28]:
tools = obj_retriever.retrieve(
    "Tell me about the eval dataset used in MetaGPT and SWE-Bench"
)

In [29]:
tools[2].metadata

ToolMetadata(description='Useful for summarization questions related to longlora', name='summary_tool_longlora', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [30]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [31]:
response = agent.query(
    "Tell me about the evaluation dataset used "
    "in MetaGPT and compare it against SWE-Bench"
)
print(str(response))

Added user message to memory: Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation dataset used in MetaGPT"}
=== Function Output ===
The evaluation dataset used in MetaGPT is the SoftwareDev dataset, which consists of 70 diverse software development tasks. These tasks range from creating Python GUI apps for drawing images, implementing color meters, and developing games like Snake, Brick breaker, 2048, Flappy bird, and Tank battle, to tasks related to Excel data processing, CRUD management, music transcription, custom press releases, Gomoku game, and weather dashboard development.
=== Calling Function ===
Calling function: summary_tool_selfrag with args: {"input": "evaluation dataset used in SWE-Bench"}
=== Function Output ===
The evaluation dataset used in SWE-Bench is not explicitly mentioned in the provided context information.
=== LLM Response ===
The ev

In [32]:
response = agent.query(
    "Compare and contrast the LoRA papers (LongLoRA, LoftQ). "
    "Analyze the approach in each paper first. "
)

Added user message to memory: Compare and contrast the LoRA papers (LongLoRA, LoftQ). Analyze the approach in each paper first. 
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "LongLoRA paper"}
=== Function Output ===
The LongLoRA paper introduces an efficient fine-tuning approach for extending the context length of large language models. It utilizes Shifted Sparse Attention (S2-Attn) during training to approximate standard self-attention patterns, enabling the extension of context lengths with reduced GPU memory cost and training time compared to full fine-tuning. LongLoRA combines trainable normalization and embedding layers to bridge the gap between low-rank adaptation (LoRA) and full fine-tuning, demonstrating its effectiveness in extending context lengths for improved performance, particularly in question-answering tasks.
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "LoftQ paper"}
=== Function Output =